In [32]:
# https://habr.com/ru/post/347564/.
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import keras
import sys
import time
#from google.colab import files # для импорта данных в google colab
#from google.colab import drive # для импорта данных в google colab из google drive
import zipfile # для работы с архивами 
from tqdm import tqdm_notebook as tqdm

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import load_model  



In [33]:
class_names_lights   = ['1', '2', '3', '4', '5']
class_names_signal   = ['0', '1', '2', '3', '4', '5']
class_names_position = ['1', '2', '3', '4', '5']

In [34]:
nb_validation_samples = 12630
batch_size = 1
n_classes_lights = 5
n_classes_signal = 6
n_classes_position = 5
validation_data_dir_lights = "/home/z_andrei/datasets/DTLD/val/lights"
validation_data_dir_signal = "/home/z_andrei/datasets/DTLD/val/colours"
validation_data_dir_position = "/home/z_andrei/datasets/DTLD/val/position"

In [35]:
img_width, img_height = 71, 71

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
validation_generator_lights = test_datagen.flow_from_directory(
    validation_data_dir_lights,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_signal = test_datagen.flow_from_directory(
    validation_data_dir_signal,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_position = test_datagen.flow_from_directory(
    validation_data_dir_position,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generators = [validation_generator_lights,\
                         validation_generator_signal,\
                         validation_generator_position]

Found 50107 images belonging to 5 classes.
Found 50107 images belonging to 6 classes.
Found 50107 images belonging to 5 classes.


In [36]:
model_lights = load_model('weights-lights-improvement-71-11-0.98.hdf5')
model_signal = load_model('weights-improvement-71-06-0.95.hdf5')
model_position = load_model('weights-position-improvement-71-11-0.98.hdf5')

In [37]:
evaluation = np.zeros((3, 3))
models = [model_lights, model_signal, model_position]
for j in tqdm(range(3)):
    evaluation[j] = models[j].evaluate_generator(validation_generators[j], verbose=1)
print(evaluation)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  ...
    to  
  ['...']
50107/50107 [==============================] - 757s 15ms/step - loss: 0.0566 - acc: 0.9827 - mse: 0.0053
  ...
    to  
  ['...']
50107/50107 [==============================] - 753s 15ms/step - loss: 0.1286 - acc: 0.9633 - mse: 0.0094
  ...
    to  
  ['...']
50107/50107 [==============================] - 562s 11ms/step - loss: 0.0768 - acc: 0.9755 - mse: 0.0074

[[0.05659848 0.98273695 0.00531896]
 [0.12859004 0.96329856 0.00940012]
 [0.07681706 0.97545254 0.00743475]]


In [38]:
predictions = list()
fps_71 = list()
for j in tqdm(range(3)):
    tic = time.perf_counter()
    predictions.append(models[j].predict_generator(validation_generators[j], verbose=1))
    toc = time.perf_counter()
    fps_71.append(len(predictions[j])/(toc - tic))
for i in range(len(predictions)):
    predictions[i] = np.array(predictions[i])
    predictions[i] = np.argmax(predictions[i], axis=1)
labels = list()
for j in tqdm(range(3)):
    labels.append(validation_generators[j].classes)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


50107/50107 [==============================] - 325s 6ms/step



/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [39]:
metrics_71 = list()
for i in range(3):
    TP = np.zeros(np.unique(labels[i]).shape[0])
    FP = np.zeros(np.unique(labels[i]).shape[0])
    TN = np.zeros(np.unique(labels[i]).shape[0])
    FN = np.zeros(np.unique(labels[i]).shape[0])
    for label in range(np.unique(labels[i]).shape[0]):
        for p, l in zip(predictions[i], labels[i]):
            TP[label] += (label == l) & (label == p)
            FP[label] += (label != l) & (label == p)
            TN[label] += (label != l) & (label != p)
            FN[label] += (label == l) & (label != p)
    precision = TP/(TP+FP)
    recall    = TP/(TP+FN)
    metrics_71.append([precision, recall])
print(metrics_71)

[[array([0.83534137, 0.88221154, 0.98775745, 0.91875   , 0.99069413]), array([0.93905192, 0.91612581, 0.98270921, 0.8891129 , 0.99035419])], [array([0.87343153, 0.93287505, 0.92139738, 0.91990847, 0.97335521,
       0.97524715]), array([0.66184374, 0.95752629, 0.75831087, 0.84100418, 0.98552215,
       0.99290861])], [array([0.98636219, 0.76656151, 0.73781513, 0.81961471, 0.97755433]), array([0.97913106, 0.5638051 , 0.76748252, 0.7301092 , 0.9896298 ])]]


## -----------------128-------------------

In [40]:
img_width, img_height = 128, 128

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
validation_generator_lights = test_datagen.flow_from_directory(
    validation_data_dir_lights,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_signal = test_datagen.flow_from_directory(
    validation_data_dir_signal,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_position = test_datagen.flow_from_directory(
    validation_data_dir_position,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generators = [validation_generator_lights,\
                         validation_generator_signal,\
                         validation_generator_position]

Found 50107 images belonging to 5 classes.
Found 50107 images belonging to 6 classes.
Found 50107 images belonging to 5 classes.


In [41]:
model_lights = load_model('weights-lights-improvement-128-09-0.98.hdf5')
model_signal = load_model('weights-improvement-128-08-0.96.hdf5')
model_position = load_model('weights-position-improvement-128-09-0.98.hdf5')

In [42]:
evaluation = np.zeros((3, 3))
models = [model_lights, model_signal, model_position]
for j in tqdm(range(3)):
    evaluation[j] = models[j].evaluate_generator(validation_generators[j], verbose=1)
print(evaluation)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  ...
    to  
  ['...']
50107/50107 [==============================] - 679s 14ms/step - loss: 0.0542 - acc: 0.9838 - mse: 0.0050
  ...
    to  
  ['...']
50107/50107 [==============================] - 733s 15ms/step - loss: 0.1213 - acc: 0.9646 - mse: 0.0089
  ...
    to  
  ['...']
50107/50107 [==============================] - 648s 13ms/step - loss: 0.0527 - acc: 0.9848 - mse: 0.0048

[[0.05418581 0.98383456 0.00496866]
 [0.1213359  0.96459579 0.00893283]
 [0.05273383 0.98477256 0.00475897]]


In [43]:
predictions = list()
fps_128 = list()
for j in tqdm(range(3)):
    tic = time.perf_counter()
    predictions.append(models[j].predict_generator(validation_generators[j], verbose=1))
    toc = time.perf_counter()
    fps_128.append(len(predictions[j])/(toc - tic))
for i in range(len(predictions)):
    predictions[i] = np.array(predictions[i])
    predictions[i] = np.argmax(predictions[i], axis=1)
labels = list()
for j in tqdm(range(3)):
    labels.append(validation_generators[j].classes)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


50107/50107 [==============================] - 463s 9ms/step



/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [44]:
metrics_128 = list()
for i in range(3):
    TP = np.zeros(np.unique(labels[i]).shape[0])
    FP = np.zeros(np.unique(labels[i]).shape[0])
    TN = np.zeros(np.unique(labels[i]).shape[0])
    FN = np.zeros(np.unique(labels[i]).shape[0])
    for label in range(np.unique(labels[i]).shape[0]):
        for p, l in zip(predictions[i], labels[i]):
            TP[label] += (label == l) & (label == p)
            FP[label] += (label != l) & (label == p)
            TN[label] += (label != l) & (label != p)
            FN[label] += (label == l) & (label != p)
    precision = TP/(TP+FP)
    recall    = TP/(TP+FN)
    metrics_128.append([precision, recall])
print(metrics_128)

[[array([0.82680412, 0.90904523, 0.9870248 , 0.85278276, 0.99266868]), array([0.90519187, 0.90314528, 0.98466756, 0.95766129, 0.99115483])], [array([0.78147448, 0.96869615, 0.9458897 , 0.9032967 , 0.94365246,
       0.99491341]), array([0.85448532, 0.93331489, 0.81671159, 0.85983264, 0.99501582,
       0.9768958 ])], [array([0.99102052, 0.57956449, 0.85798817, 0.94715447, 0.99184389]), array([0.99275884, 0.80278422, 0.76048951, 0.72698908, 0.99218422])]]


## -----------------200-------------------

In [45]:
img_width, img_height = 200, 200

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
validation_generator_lights = test_datagen.flow_from_directory(
    validation_data_dir_lights,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_signal = test_datagen.flow_from_directory(
    validation_data_dir_signal,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_position = test_datagen.flow_from_directory(
    validation_data_dir_position,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generators = [validation_generator_lights,\
                         validation_generator_signal,\
                         validation_generator_position]

Found 50107 images belonging to 5 classes.
Found 50107 images belonging to 6 classes.
Found 50107 images belonging to 5 classes.


In [46]:
model_lights = load_model('weights-lights-improvement-12-0.98.hdf5')
model_signal = load_model('weights-improvement-12-0.97.hdf5')
model_position = load_model('weights-position-improvement-12-0.98.hdf5')

In [47]:
evaluation = np.zeros((3, 3))
models = [model_lights, model_signal, model_position]
for j in tqdm(range(3)):
    evaluation[j] = models[j].evaluate_generator(validation_generators[j], verbose=1)
print(evaluation)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  ...
    to  
  ['...']
50107/50107 [==============================] - 744s 15ms/step - loss: 0.0761 - acc: 0.9755 - mse: 0.0074
  ...
    to  
  ['...']
50107/50107 [==============================] - 931s 19ms/step - loss: 0.0713 - acc: 0.9812 - mse: 0.0050
  ...
    to  
  ['...']
50107/50107 [==============================] - 747s 15ms/step - loss: 0.0454 - acc: 0.9866 - mse: 0.0041

[[0.07606523 0.97547251 0.00737488]
 [0.07133942 0.98118025 0.0049918 ]
 [0.04540233 0.98656875 0.00411868]]


In [48]:
predictions = list()
fps_200 = list()
for j in tqdm(range(3)):
    tic = time.perf_counter()
    predictions.append(models[j].predict_generator(validation_generators[j], verbose=1))
    toc = time.perf_counter()
    fps_200.append(len(predictions[j])/(toc - tic))
for i in range(len(predictions)):
    predictions[i] = np.array(predictions[i])
    predictions[i] = np.argmax(predictions[i], axis=1)
labels = list()
for j in tqdm(range(3)):
    labels.append(validation_generators[j].classes)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


50107/50107 [==============================] - 601s 12ms/step



/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [49]:
metrics_200 = list()
for i in range(3):
    TP = np.zeros(np.unique(labels[i]).shape[0])
    FP = np.zeros(np.unique(labels[i]).shape[0])
    TN = np.zeros(np.unique(labels[i]).shape[0])
    FN = np.zeros(np.unique(labels[i]).shape[0])
    for label in range(np.unique(labels[i]).shape[0]):
        for p, l in zip(predictions[i], labels[i]):
            TP[label] += (label == l) & (label == p)
            FP[label] += (label != l) & (label == p)
            TN[label] += (label != l) & (label != p)
            FN[label] += (label == l) & (label != p)
    precision = TP/(TP+FP)
    recall    = TP/(TP+FN)
    metrics_200.append([precision, recall])
print(metrics_200)

[22074.   328.   515.   554. 25963.] [217. 103. 193.  56. 104.] [160. 103.  57.  87. 266.] [27656. 49573. 49342. 49410. 23774.] [22234.   431.   572.   641. 26229.]
[[array([0.99026513, 0.76102088, 0.72740113, 0.90819672, 0.99601028]), array([0.99280381, 0.76102088, 0.90034965, 0.86427457, 0.98985855])]]


In [50]:
print(fps_71, fps_128, fps_200)

[162.583433362638, 162.55573402503316, 154.25195591471044] [135.498097935847, 90.91535739182729, 108.23930500146126] [77.1401153895377, 114.27808354486719, 83.35116122189298]
